In [26]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps, operator_average
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t_spin, get_h1_spin, get_g_spin, get_t
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt,  generate_krylov_space_othogonal_against
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
from pytenet.operation_thc import get_num_op_mpo


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [27]:
#load integrals
with h5py.File("/work_fast/ge49cag/code_datas/NH3/integral.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/NH3/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

8
(8, 36)
(36, 36)
rl errV: 6.421105521729929e-13
abs errV: 4.286591936668418e-12
errt: 1.0574710399920436e-13
errh: 1.8088404989173088e-14
errht: 9.006422291220954e-14


In [28]:
# H_correct = get_molecular_Hamiltonian_as_sparse_matrix(get_h1_spin(h1), get_g_spin(eri))
# H_correct = scipy.io.mmread('/work_fast/ge49cag/code_datas/H_correct_NH3.mtx').tocsr()
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

In [29]:
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [30]:
e_ground = -67.46923609481743
e_1st_ex = -67.00839085980371
e_2nd_ex = -67.00038986115365

We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [31]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 62, 100, 154, 100, 62, 16, 1]


Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [32]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state.

In [33]:
filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_1st_excited.pkl"
with open(filename, 'rb') as file:
    initial = pickle.load(file)

print(operator_average(initial, mpo_ref) - e_1st_ex)

(0.00929712659700499+0j)


In [34]:
filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_ground_state.pkl"
with open(filename, 'rb') as file:
    NH3_ground = pickle.load(file)
    
print(operator_average(NH3_ground, mpo_ref) - e_ground)    

(2.9257942912863655e-07+0j)


MAYBE: for each new-generated Krylov, we delete the component of ground state and then store it.
When approximate state is not available, orthogonalize a initial state to all previous ground/excited states.
实在不行：用HFS强行和基态正交再找激发态。

generate Krylov space:

In [35]:
N_Krylov_1 = 30
psi_original_1 = copy.deepcopy(initial)
max_bond_Krylov_1 = 150
trunc_tol = 0
foldername_1 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex" 
#generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)
generate_krylov_space_othogonal_against(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1, copy.deepcopy(NH3_ground))
#generate_krylov_space_othogonal_against(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername, vec_to_remove)

[1, 4, 16, 64, 130, 64, 16, 4, 1]
2
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.0067494797885712e-16+0j)
3
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(2.314771638256552e-17+0j)
4
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-4.2392304944183223e-17+0j)
5
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.6686549060909717e-17+0j)
6
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.0222135558033436e-17+0j)
7
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(2.008484524529397e-17+0j)
8
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.9109063290057016e-18+0j)
9
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.918713227177295e-17+0j)
10
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.485709984540897e-17+0j)
11
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(8.87148427636264e-17+0j)
12
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.8675382421062814e-17+0j)
13
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-2.4048959438444095e-17+0j)
14
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.734723475976807e-18+0j)
15
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+

In [36]:
#或许在这一步生成GS的时候也把ground state考虑进来
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)


/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [37]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_1st_ex, mpo_ref)

(0.007273515595201729+0j)
(0.004818042063845951+0j)
(0.00280449602921351+0j)
(0.001728308915247112+0j)
(0.0011594267369332556+0j)
(0.0009534374583637373+0j)


In [39]:
# spin = 0
# for i in range (L):
#     if i%2 == 0:
#         spin_local = 0.5* operator_average(v_ritz_1, get_num_op_mpo(i, L))
#     if i%2 == 1:
#         spin_local = -0.5* operator_average(v_ritz_1, get_num_op_mpo(i, L))
#     spin += spin_local 
# print(spin)

Restart:

In [40]:
N_Krylov_2 = 30
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 150
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_1"
generate_krylov_space_othogonal_against(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2, copy.deepcopy(NH3_ground))

[1, 4, 16, 64, 130, 64, 16, 4, 1]
2
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(6.836572323878909e-17+0j)
3
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(4.294118229400401e-17+0j)
4
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(6.085084693074894e-18+0j)
5
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.3377699555755518e-16+0j)
6
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(8.683442962072185e-17+0j)
7
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-2.879234394306818e-17+0j)
8
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.1153729849444627e-17+0j)
9
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-5.611085055791387e-17+0j)
10
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.1140177322288558e-17+0j)
11
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.3806778990813635e-17+0j)
12
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-7.218075963322246e-17+0j)
13
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.5385506453927111e-16+0j)
14
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-6.299214622140781e-17+0j)
15
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+

In [41]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)
#H_reduced_2 = remain_only_tridiagonal_elements(H_reduced_2)

In [42]:
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)

In [43]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_1st_ex, mpo_ref)

(0.0009128678030378978+0j)
(0.0007946191481380538+0j)
(0.0006629588337574432+0j)
(0.0005669537185752915+0j)
(0.0004905346178674108+0j)


(0.0003954864077968523+0j)


Restart:

In [44]:
N_Krylov_3 = 20
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 150
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_2"
generate_krylov_space_othogonal_against(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3, copy.deepcopy(NH3_ground))


[1, 4, 16, 64, 130, 64, 16, 4, 1]
2
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-9.147955830346444e-19+0j)
3
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(2.522802930102208e-17+0j)
4
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(5.129631528572043e-17+0j)
5
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.2183721913305856e-17+0j)
6
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-9.550465886881687e-17+0j)
7
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.4071927759325142e-16+0j)
8
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-8.030549966328571e-17+0j)
9
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-5.6107462426124854e-18+0j)
10
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.322049024074512e-17+0j)
11
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(2.15485181781494e-17+0j)
12
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-4.0345873343616834e-17+0j)
13
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-3.5948078281472506e-17+0j)
14
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(5.283452711793424e-17+0j)
15
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+

In [45]:
H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)


In [46]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_1st_ex, mpo_ref)

(0.00034600424542929886+0j)
(0.00030049075907356837+0j)


(0.0002538725854748236+0j)
(0.00021170241001300383+0j)


restart:

In [47]:
N_Krylov_4 = 20
psi_original_4 = copy.deepcopy(v_ritz_3)
max_bond_Krylov_4 = 150
trunc_tol = 0
foldername_4 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_3"
generate_krylov_space_othogonal_against(N_Krylov_4, H_mu_nu_list_spin_layer, psi_original_4, max_bond_Krylov_4, trunc_tol, r_thc, foldername_4, copy.deepcopy(NH3_ground))


[1, 4, 16, 64, 130, 64, 16, 4, 1]
2
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(2.303929616531697e-17+0j)
3
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.0330413824713447e-16+0j)
4
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-2.0684544571950014e-17+0j)
5
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.2228106439741981e-16+0j)
6
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(6.8406380820257295e-18+0j)
7
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-4.496050884025826e-17+0j)
8
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(5.2932782939815737e-17+0j)
9
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(5.2292426031691486e-17+0j)
10
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.416592096044946e-17+0j)
11
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(2.0850563029611857e-17+0j)
12
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.620882247865829e-17+0j)
13
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(2.9449641510137514e-17+0j)
14
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.519238294195313e-16+0j)
15
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1

In [48]:
H_reduced_non_rotho_4 = generate_reduced_H_non_ortho(N_Krylov_4, foldername_4, mpo_ref)
coeff_4 = coeff_gram_schmidt(N_Krylov_4, foldername_4)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_4 = np.einsum('ik, kl, jl -> ij', coeff_4.conj(), H_reduced_non_rotho_4, coeff_4)

In [49]:
e_ritz_4, v_ritz_4 = solve_ritz(foldername_4, H_reduced_4, N_Krylov_4, coeff_4, max_bond_Krylov_4, e_1st_ex, mpo_ref)

(0.0001957093503079932+0j)
(0.00018190127475747886+0j)
(0.00016519590333530232+0j)
(0.00013315061545426943+0j)


restart:

In [50]:
N_Krylov_5 = 50
psi_original_5 = copy.deepcopy(v_ritz_4)
max_bond_Krylov_5 = 150
trunc_tol = 0
foldername_5 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_4"
generate_krylov_space_othogonal_against(N_Krylov_5, H_mu_nu_list_spin_layer, psi_original_5, max_bond_Krylov_5, trunc_tol, r_thc, foldername_5, copy.deepcopy(NH3_ground))

[1, 4, 16, 64, 130, 64, 16, 4, 1]
2
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.4206578541917665e-17+0j)
3
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.8133281334820062e-17+0j)
4
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-5.244828009398628e-18+0j)
5
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-6.450325299930948e-17+0j)
6
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-8.257554796192723e-17+0j)
7
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-6.391371806802049e-17+0j)
8
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.2603850255143989e-17+0j)
9
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(4.0766001685454967e-17+0j)
10
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(8.755948982357153e-17+0j)
11
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(1.1207939958068902e-17+0j)
12
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(6.095926714799749e-17+0j)
13
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-1.2130528244218286e-16+0j)
14
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.4206578541917665e-17+0j)
15
[1, 4, 16, 64, 130, 64, 16, 4, 1]


In [51]:
H_reduced_non_rotho_5 = generate_reduced_H_non_ortho(N_Krylov_5, foldername_5, mpo_ref)
coeff_5 = coeff_gram_schmidt(N_Krylov_5, foldername_5)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_5 = np.einsum('ik, kl, jl -> ij', coeff_5.conj(), H_reduced_non_rotho_5, coeff_5)

In [52]:
e_ritz_5, v_ritz_5 = solve_ritz(foldername_5, H_reduced_5, N_Krylov_5, coeff_5, max_bond_Krylov_5, e_1st_ex, mpo_ref)

(0.00011792875700677996+0j)
(0.00010461903880809587+0j)


(9.648781013993357e-05+0j)
(8.659310577741053e-05+0j)
(5.353524180407021e-05+0j)
(3.7816789188127586e-05+0j)
(2.0977322321868996e-05+0j)
(1.1811459827981707e-05+0j)
(3.860821337298148e-06+0j)
(9.781012693110824e-07+0j)


In [60]:
N_Krylov_6 = 50
psi_original_6 = copy.deepcopy(v_ritz_5)
max_bond_Krylov_6 = 150
trunc_tol = 0
foldername_6 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_5"
generate_krylov_space_othogonal_against(N_Krylov_6, H_mu_nu_list_spin_layer, psi_original_6, max_bond_Krylov_6, trunc_tol, r_thc, foldername_6, copy.deepcopy(NH3_ground))

[1, 4, 16, 64, 130, 64, 16, 4, 1]
2
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.3745792618611326e-18+0j)
3
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-4.363913744254155e-18+0j)
4
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-9.356664748549903e-17+0j)
5
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(7.521313758439285e-17+0j)
6
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-3.531788576871531e-17+0j)
7
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(2.5139937874507634e-17+0j)
8
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-2.3493305825045274e-17+0j)
9
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-8.353777739000812e-17+0j)
10
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(3.4436971503570835e-17+0j)
11
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(9.161508357502512e-17+0j)
12
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-9.904186845655083e-17+0j)
13
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(-5.603969979034451e-17+0j)
14
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1+0j)
(4.6702008579813103e-17+0j)
15
[1, 4, 16, 64, 130, 64, 16, 4, 1]
(1

In [61]:
H_reduced_non_rotho_6 = generate_reduced_H_non_ortho(N_Krylov_6, foldername_6, mpo_ref)
coeff_6 = coeff_gram_schmidt(N_Krylov_6, foldername_6)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_6 = np.einsum('ik, kl, jl -> ij', coeff_6.conj(), H_reduced_non_rotho_6, coeff_6)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [62]:
e_ritz_6, v_ritz_6 = solve_ritz(foldername_6, H_reduced_6, N_Krylov_6, coeff_6, max_bond_Krylov_6, e_1st_ex, mpo_ref)

(6.142353470295348e-07+0j)
(5.389562574009688e-07+0j)
(5.135676417467039e-07+0j)
(4.959903492363082e-07+0j)
(4.883235931174568e-07+0j)
(4.873014063377923e-07+0j)
(4.842666498916515e-07+0j)
(4.822072270371791e-07+0j)
(4.819374055387016e-07+0j)
(4.808381532939165e-07+0j)
